# Canvas Animation Extractor for Wuthering Waves - Google Colab Version\n
\n
This notebook extracts animated canvas content from web pages and converts them to high-quality MP4 videos.\n
\n
**Features:**\n
- Web scraping with Selenium and Chrome\n
- Canvas animation frame capture\n
- High-resolution image processing with PIL\n
- Video conversion with FFmpeg\n
- Optimized for Google Colab environment\n
\n
**Target Website:** Wuthering Waves Event Page

## Step 1: Install Required Dependencies\n
\n
First, we'll install all the necessary packages for web scraping, image processing, and video conversion.

In [ ]:
# Install required packages\n
!apt-get update\n
!apt-get install -y chromium-driver ffmpeg\n
\n
# Install Python packages\n
!pip install selenium pillow numpy opencv-python\n
\n
# Install additional dependencies for video processing\n
!pip install imageio imageio-ffmpeg\n
\n
print('All dependencies installed successfully!')

## Step 2: Import Required Libraries\n
\n
Import all the necessary libraries for web scraping, image processing, and video creation.

In [ ]:
import os\n
import time\n
import base64\n
import json\n
from datetime import datetime\n
from pathlib import Path\n
\n
# Web scraping and browser automation\n
from selenium import webdriver\n
from selenium.webdriver.chrome.options import Options\n
from selenium.webdriver.common.by import By\n
from selenium.webdriver.support.ui import WebDriverWait\n
from selenium.webdriver.support import expected_conditions as EC\n
\n
# Image processing\n
from PIL import Image\n
import numpy as np\n
import cv2\n
\n
# Video creation\n
import imageio\n
\n
print('All libraries imported successfully!')\n
print(f'Current time: {datetime.now().strftime(\"%Y-%m-%d %H:%M:%S\")}')

## Step 3: Configuration Settings\n
\n
Set up the configuration parameters for canvas capture and video creation.

In [ ]:
# Configuration settings\n
CONFIG = {\n
    'url': 'https://wutheringwaves-event1.kurogames-global.com/?packageId=A1730&language=en&isInternalBrowser=0&platform=PC',\n
    'max_frames': 600,\n
    'capture_fps': 30,\n
    'output_fps': 30,\n
    'viewport_width': 3840,\n
    'viewport_height': 2160,\n
    'wait_time': 10,\n
    'frame_interval': 1.0/30,\n
    'output_dir': 'canvas_frames',\n
    'video_name': 'wuthering_waves_animation.mp4'\n
}\n
\n
# Create output directory\n
os.makedirs(CONFIG['output_dir'], exist_ok=True)\n
\n
print('Configuration loaded:')\n
for key, value in CONFIG.items():\n
    print(f'   {key}: {value}')

## Step 4: Browser Setup Functions\n
\n
Create functions to set up Chrome browser with optimal settings for canvas capture.

In [ ]:
def setup_chrome_driver():\n
    chrome_options = Options()\n
    \n
    # Essential Chrome options for Colab environment\n
    chrome_options.add_argument('--headless')\n
    chrome_options.add_argument('--no-sandbox')\n
    chrome_options.add_argument('--disable-dev-shm-usage')\n
    chrome_options.add_argument('--disable-gpu')\n
    chrome_options.add_argument('--disable-features=VizDisplayCompositor')\n
    chrome_options.add_argument('--disable-web-security')\n
    chrome_options.add_argument('--allow-running-insecure-content')\n
    chrome_options.add_argument('--disable-blink-features=AutomationControlled')\n
    \n
    # Set window size for high resolution capture\n
    chrome_options.add_argument(f'--window-size={CONFIG[\"viewport_width\"]},{CONFIG[\"viewport_height\"]}')\n
    \n
    # Performance optimizations\n
    chrome_options.add_argument('--disable-extensions')\n
    chrome_options.add_argument('--disable-plugins')\n
    \n
    try:\n
        driver = webdriver.Chrome(options=chrome_options)\n
        driver.implicitly_wait(10)\n
        driver.set_page_load_timeout(60)\n
        print('Chrome driver setup successfully!')\n
        return driver\n
    except Exception as e:\n
        print(f'Failed to setup Chrome driver: {e}')\n
        return None\n
\n
print('Browser setup function ready!')

## Step 5: Canvas Capture Functions\n
\n
Create functions to detect and capture canvas animations from the web page.

In [ ]:
def detect_canvas_elements(driver):\n
    try:\n
        canvas_elements = WebDriverWait(driver, 20).until(\n
            EC.presence_of_all_elements_located((By.TAG_NAME, 'canvas'))\n
        )\n
        \n
        canvas_info = []\n
        for i, canvas in enumerate(canvas_elements):\n
            try:\n
                info = driver.execute_script(\"\"\"\n
                    var canvas = arguments[0];\n
                    return {\n
                        width: canvas.width,\n
                        height: canvas.height,\n
                        offsetWidth: canvas.offsetWidth,\n
                        offsetHeight: canvas.offsetHeight,\n
                        id: canvas.id,\n
                        className: canvas.className\n
                    };\n
                \"\"\", canvas)\n
                info['index'] = i\n
                info['element'] = canvas\n
                canvas_info.append(info)\n
            except Exception as e:\n
                print(f'Error getting info for canvas {i}: {e}')\n
        \n
        return canvas_info\n
    except Exception as e:\n
        print(f'Error detecting canvas elements: {e}')\n
        return []\n
\n
def capture_canvas_frame(driver, canvas_element):\n
    try:\n
        canvas_data = driver.execute_script(\"\"\"\n
            var canvas = arguments[0];\n
            try {\n
                return canvas.toDataURL('image/png');\n
            } catch (e) {\n
                return null;\n
            }\n
        \"\"\", canvas_element)\n
        \n
        if canvas_data and canvas_data.startswith('data:image/png;base64,'):\n
            return canvas_data\n
        else:\n
            return None\n
    except Exception as e:\n
        return None\n
\n
def save_frame(canvas_data, frame_number, output_dir):\n
    try:\n
        base64_data = canvas_data.split(',')[1]\n
        image_data = base64.b64decode(base64_data)\n
        \n
        filename = f'frame_{frame_number:06d}.png'\n
        filepath = os.path.join(output_dir, filename)\n
        \n
        with open(filepath, 'wb') as f:\n
            f.write(image_data)\n
        \n
        return filepath\n
    except Exception as e:\n
        print(f'Error saving frame {frame_number}: {e}')\n
        return None\n
\n
print('Canvas capture functions ready!')

## Step 6: Main Canvas Animation Capture\n
\n
The main function that orchestrates the entire canvas capture process.

In [ ]:
def capture_canvas_animation():\n
    print('Starting canvas animation capture...')\n
    print(f'Target URL: {CONFIG[\"url\"]}')\n
    \n
    driver = setup_chrome_driver()\n
    if not driver:\n
        return False, []\n
    \n
    try:\n
        print('Loading website...')\n
        driver.get(CONFIG['url'])\n
        \n
        print(f'Waiting {CONFIG[\"wait_time\"]} seconds for page load...')\n
        time.sleep(CONFIG['wait_time'])\n
        \n
        print('Detecting canvas elements...')\n
        canvas_info = detect_canvas_elements(driver)\n
        \n
        if not canvas_info:\n
            print('No canvas elements found!')\n
            return False, []\n
        \n
        print(f'Found {len(canvas_info)} canvas elements:')\n
        for info in canvas_info:\n
            print(f'   Canvas {info[\"index\"]}: {info[\"width\"]}x{info[\"height\"]}')\n
        \n
        target_canvas = max(canvas_info, key=lambda x: x['width'] * x['height'])\n
        print(f'Selected canvas {target_canvas[\"index\"]} for capture')\n
        \n
        print('Starting frame capture...')\n
        captured_frames = []\n
        frame_count = 0\n
        consecutive_failures = 0\n
        \n
        start_time = time.time()\n
        \n
        while frame_count < CONFIG['max_frames'] and consecutive_failures < 10:\n
            canvas_data = capture_canvas_frame(driver, target_canvas['element'])\n
            \n
            if canvas_data:\n
                filepath = save_frame(canvas_data, frame_count, CONFIG['output_dir'])\n
                if filepath:\n
                    captured_frames.append(filepath)\n
                    consecutive_failures = 0\n
                    \n
                    if frame_count % 30 == 0:\n
                        elapsed = time.time() - start_time\n
                        print(f'Captured {frame_count + 1} frames ({elapsed:.1f}s elapsed)')\n
                else:\n
                    consecutive_failures += 1\n
            else:\n
                consecutive_failures += 1\n
            \n
            frame_count += 1\n
            time.sleep(CONFIG['frame_interval'])\n
        \n
        total_time = time.time() - start_time\n
        print(f'Capture completed! Total frames: {len(captured_frames)}')\n
        print(f'Total time: {total_time:.1f}s')\n
        \n
        return len(captured_frames) > 0, captured_frames\n
    \n
    except Exception as e:\n
        print(f'Error during capture: {e}')\n
        return False, []\n
    \n
    finally:\n
        driver.quit()\n
        print('Browser closed')\n
\n
print('Main capture function ready!')

## Step 7: Video Conversion Functions\n
\n
Convert the captured frames into a high-quality MP4 video.

In [ ]:
def create_video_from_frames(frame_files, output_path, fps=30):\n
    if not frame_files:\n
        print('No frames to convert!')\n
        return False\n
    \n
    try:\n
        print(f'Creating video from {len(frame_files)} frames...')\n
        print(f'Output: {output_path}')\n
        print(f'FPS: {fps}')\n
        \n
        frame_files.sort()\n
        \n
        first_frame = cv2.imread(frame_files[0])\n
        if first_frame is None:\n
            print('Could not read first frame!')\n
            return False\n
        \n
        height, width, layers = first_frame.shape\n
        print(f'Video dimensions: {width}x{height}')\n
        \n
        with imageio.get_writer(output_path, fps=fps, quality=9, codec='libx264') as writer:\n
            for i, frame_file in enumerate(frame_files):\n
                frame = cv2.imread(frame_file)\n
                if frame is not None:\n
                    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)\n
                    writer.append_data(frame_rgb)\n
                    \n
                    if i % 30 == 0:\n
                        progress = (i + 1) / len(frame_files) * 100\n
                        print(f'Progress: {progress:.1f}%')\n
        \n
        print(f'Video created successfully: {output_path}')\n
        file_size = os.path.getsize(output_path) / (1024 * 1024)\n
        print(f'File size: {file_size:.1f} MB')\n
        \n
        return True\n
        \n
    except Exception as e:\n
        print(f'Error creating video: {e}')\n
        return False\n
\n
print('Video conversion functions ready!')

## Step 8: Execute Canvas Animation Capture\n
\n
Run the complete canvas animation capture and video creation process.

In [ ]:
# Main execution\n
print('=' * 60)\n
print('CANVAS ANIMATION EXTRACTOR FOR WUTHERING WAVES')\n
print('=' * 60)\n
\n
# Step 1: Capture canvas animation\n
success, captured_frames = capture_canvas_animation()\n
\n
if success and captured_frames:\n
    print(f'Successfully captured {len(captured_frames)} frames!')\n
    \n
    # Step 2: Create video from frames\n
    print('Creating video...')\n
    video_output = CONFIG['video_name']\n
    \n
    video_success = create_video_from_frames(\n
        captured_frames, \n
        video_output, \n
        fps=CONFIG['output_fps']\n
    )\n
    \n
    if video_success:\n
        print(f'SUCCESS! Video created: {video_output}')\n
        print('You can download the video file from the Colab file browser!')\n
    else:\n
        print('Video creation failed!')\n
else:\n
    print('Canvas capture failed!')\n
\n
print('Process completed!')